In [65]:
# Стандартизация
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('data/occupancy_even.csv')

scaler = preprocessing.StandardScaler()

names = df.columns
scaled_data = scaler.fit_transform(df)

df_std = pd.DataFrame(scaled_data, columns=names)


In [66]:
# Создание и обучение классификатора
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

x = df_std.iloc[:, :-1].values  # все колонки до последней - переменные функции
y = df.iloc[:, -1].values  # в качестве результирующего значения берем последнюю колонку

# делим набор данных на обучающий и тестовый
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

classifier = tree.DecisionTreeClassifier()
classifier.fit(x_train, y_train)  # идентифицирует и изучает параметры модели из обучающего набора данных
# tree.plot_tree(classifier)

DecisionTreeClassifier()

In [67]:
# Рисуем график
import graphviz

dot_data = tree.export_graphviz(classifier,
                                out_file=None,
                                feature_names=df.columns[:-1],
                                class_names=['Unoccupied', 'Occupied'],
                                filled=True,
                                rounded=True,
                                special_characters=True
                                )
graph = graphviz.Source(dot_data)
graph.render('output/tree')

'output/tree.pdf'

In [68]:
# Оценка качества классификатора
y_pred = classifier.predict(x_test)  # прогноз на основе тестовых данных

# классификационная оценка точности
print("Точность:\n", accuracy_score(y_test, y_pred), "\n")

# вычисляет матрицу ошибок для оценки точности классификации
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred), "\n")


Точность:
 0.9821073558648111 

Матрица ошибок:
 [[246   2]
 [  7 248]] 



In [69]:
# Генератор входных данных
import pandas as pd
import numpy as np

n = 15  # количество входных объектов

# генерируем параметры в диапазоне, большем допустимого интервала
temp = np.random.uniform(low=15.0, high=30.0, size=(n,))
humid = np.random.uniform(low=10.0, high=40.0, size=(n,))
light = np.random.uniform(low=0.0, high=1200.0, size=(n,))
co2 = np.random.uniform(low=300.0, high=2000.0, size=(n,))
humid_ratio = np.random.uniform(low=0.001, high=0.007, size=(n,))
occupancy = np.zeros((n,))

matrix_aux = np.vstack([temp, humid, light, co2, humid_ratio, occupancy])
matrix = np.transpose(matrix_aux)
df_input = pd.DataFrame(matrix, columns=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'])

print(df_input)


    Temperature   Humidity        Light          CO2  HumidityRatio  Occupancy
0     25.007822  13.807833  1091.602204  1504.316816       0.006373        0.0
1     18.107378  34.765113  1056.903239  1162.647435       0.004068        0.0
2     20.091952  38.389722  1107.109418   908.777899       0.002757        0.0
3     23.704244  24.855881   387.038581  1505.315505       0.003040        0.0
4     28.393993  20.291510    19.504580  1781.382770       0.001856        0.0
5     16.903032  22.253616   185.051296  1886.232192       0.004324        0.0
6     18.227400  17.155895   683.806482  1782.125370       0.003370        0.0
7     24.686072  25.637688   575.171867  1423.301675       0.004758        0.0
8     18.583200  30.034793  1010.081176   308.523012       0.001267        0.0
9     29.004404  24.388549  1153.406937  1674.350474       0.006653        0.0
10    19.675989  29.713697   842.335916  1516.344011       0.005953        0.0
11    19.678864  32.095441  1146.090968   377.035730

In [70]:
# Стандартизируем тестовые данные
import pandas as pd

input_scaled = scaler.transform(df_input)

df_input_std = pd.DataFrame(input_scaled, columns=names)
print(df_input_std)


    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
0      3.140746 -4.754112  3.305640  2.394265       3.531204  -0.963873
1     -3.245960  3.608663  3.169574  1.274109      -0.103137  -0.963873
2     -1.409137  5.055024  3.366448  0.441803      -2.168798  -0.963873
3      1.934220 -0.345509  0.542816  2.397540      -1.722717  -0.963873
4      6.274817 -2.166872 -0.898405  3.302620      -3.589527  -0.963873
5     -4.360642 -1.383914 -0.249242  3.646367       0.301180  -0.963873
6     -3.134873 -3.418104  1.706540  3.305055      -1.202519  -0.963873
7      2.842950 -0.033537  1.280548  2.128659       0.985750  -0.963873
8     -2.805563  1.721080  2.985969 -1.526120      -4.517788  -0.963873
9      6.839783 -0.531992  3.547996  2.951717       3.971757  -0.963873
10    -1.794132  1.592950  2.328185  2.433696       2.868462  -0.963873
11    -1.791471  2.543359  3.519308 -1.301503       4.480130  -0.963873
12     0.829884  4.771423 -0.281144  1.526592       0.592261  -0

In [71]:
# Удаляем значения вне доверительного интервала
std = df_std.std()

for_deletion = df_input_std.index[
    (df_input_std.Temperature < (-3 * std.Temperature)) |
    (df_input_std.Temperature > (3 * std.Temperature)) |
    (df_input_std.Humidity < (-3 * std.Humidity)) |
    (df_input_std.Humidity > (3 * std.Humidity)) |
    (df_input_std.Light < (-3 * std.Light)) |
    (df_input_std.Light > (3 * std.Light)) |
    (df_input_std.CO2 < (-3 * std.CO2)) |
    (df_input_std.CO2 > (3 * std.CO2)) |
    (df_input_std.HumidityRatio < (-3 * std.HumidityRatio)) |
    (df_input_std.HumidityRatio > (3 * std.HumidityRatio))
    ].tolist()

df_input_clean = df_input.drop(for_deletion)
df_input_clean_std = df_input_std.drop(for_deletion)

print(df_input_clean, '\n')
print(df_input_clean_std)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
3     23.704244  24.855881  387.038581  1505.315505       0.003040        0.0
7     24.686072  25.637688  575.171867  1423.301675       0.004758        0.0
10    19.675989  29.713697  842.335916  1516.344011       0.005953        0.0 

    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
3      1.934220 -0.345509  0.542816  2.397540      -1.722717  -0.963873
7      2.842950 -0.033537  1.280548  2.128659       0.985750  -0.963873
10    -1.794132  1.592950  2.328185  2.433696       2.868462  -0.963873


In [72]:
# Подаем полученные данные на вход классификатору

input_x = df_input_clean_std.iloc[:, :-1].values

input_y = classifier.predict(input_x)
probs = classifier.predict_proba(input_x)
probs = np.transpose(probs)

df_input_clean.Occupancy = input_y
df_input_clean['Probability0'] = probs[0]
df_input_clean['Probability1'] = probs[1]

print(df_input_clean)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy  \
3     23.704244  24.855881  387.038581  1505.315505       0.003040          0   
7     24.686072  25.637688  575.171867  1423.301675       0.004758          1   
10    19.675989  29.713697  842.335916  1516.344011       0.005953          0   

    Probability0  Probability1  
3            1.0           0.0  
7            0.0           1.0  
10           1.0           0.0  
